In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
359,tt0458525,150000000,341131793,X-Men Origins: Wolverine,Hugh Jackman|Liev Schreiber|Danny Huston|Lynn ...,Gavin Hood,Witness the Origin.,"After seeking to live a normal life, Logan set...",107,Adventure|Action|Thriller|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,4/28/2009,6.2,2009
1216,tt2101441,5000000,31724284,Spring Breakers,James Franco|Selena Gomez|Vanessa Hudgens|Ashl...,Harmony Korine,A little sun can bring out your dark side.,After four college girls rob a restaurant to f...,94,Drama|Crime,RabbitBandini Productions|Division Films|Pop F...,1/24/2013,5.0,2013
109,tt3787590,6000000,11122090,We Are Your Friends,Zac Efron|Emily Ratajkowski|Wes Bentley|Nick R...,Max Joseph,The world is yours.,Young Cole Carter dreams of hitting the big ti...,96,Drama|Music|Romance,Working Title Films,8/26/2015,6.2,2015
1802,tt0162222,90000000,429632142,Cast Away,Tom Hanks|Helen Hunt|Chris Noth|Paul Sanchez|L...,Robert Zemeckis,"At the edge of the world, his journey begins.","Chuck, a top international manager for FedEx, ...",143,Adventure|Drama,DreamWorks SKG|Twentieth Century Fox Film Corp...,12/22/2000,7.3,2000
1450,tt0452594,52000000,204999686,The Break-Up,Vince Vaughn|Jennifer Aniston|Joey Lauren Adam...,Peyton Reed,. . . pick a side.,Cohabitating couple Gary and Brooke find their...,106,Romance|Comedy,Universal Pictures|Mosaic Media Group|Wild Wes...,6/1/2006,5.5,2006


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

data['profit'] = (data['revenue'] - data['budget']) # добавяем колонку 'profit' с прибылью от фильма
data['release_date'] = pd.to_datetime(data['release_date']) # преобразовываем дату релиза

display(data.sample(5)) # проверили что получилось

display(data.info(verbose=True, null_counts=True)) # проверяем информацию о колонках датафрейма

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1471,tt0427229,50000000,88715192,Failure to Launch,Matthew McConaughey|Sarah Jessica Parker|Zooey...,Tom Dey,"To leave the nest, some men just need a little...","Tripp, an attractive man in his thirties, is s...",97,Comedy,Paramount Pictures|Scott Rudin Productions|Fai...,2006-03-10,5.6,2006,38715192
1774,tt0465538,25000000,92991835,Michael Clayton,George Clooney|Tom Wilkinson|Tilda Swinton|Syd...,Tony Gilroy,The truth can be adjusted,A law firm brings in its 'fixer' to remedy the...,119,Drama|Mystery|Crime,Castle Rock Entertainment|Mirage Enterprises|S...,2007-09-28,6.3,2007,67991835
1263,tt0765446,40000000,74597643,Escape from Planet Earth,Brendan Fraser|Rob Corddry|Ricky Gervais|Jessi...,Callan Brunker,Earth's greatest secrets are about to break out!,Astronaut Scorch Supernova finds himself caugh...,89,Animation|Comedy|Adventure|Family|Science Fiction,Rainmaker Entertainment|Mainframe Entertainmen...,2013-02-14,5.8,2013,34597643
1425,tt0357507,20000000,67192859,Boogeyman,Emily Deschanel|Barry Watson|Skye McCole Bartu...,Stephen Kay,You thought it was just a story.,Every culture has one â€“ the horrible monster...,89,Thriller|Horror|Drama|Mystery,Ghost House Pictures|Senator International|Scr...,2005-02-04,4.5,2005,47192859
1412,tt0421054,50000000,22944502,Domino,Keira Knightley|Mickey Rourke|Edgar RamÃ­rez|D...,Tony Scott,Heads You Live... Tails You Die.,"The daughter of actor, Laurence Harvey turns a...",127,Action|Crime,New Line Cinema|Davis-Films|Metropolitan Filme...,2005-09-22,6.1,2005,-27055498


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

None

# 1. У какого фильма из списка самый большой бюджет?

In [5]:

i = data[data.budget == data.budget.max()] # находим фильм с максимальным бюджетом с помощью среза

'''Следующий блок кода пригодится нам многократно для более
читабельного вывода названия фильма: получив на вход строку датафрейма (i)
на выходе мы получаем вывода формата "название_фильма (идентификатор_imdb)". '''
title = str(i.original_title)    # переводим название фильма в строковую величину
imdb = str(i.imdb_id)            # переводим идентификатор imdb в строковую величину
i1 = title[title.find(' ') + 4 : title.find('\nName:')]  # "отсекаем" лишние части строки
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]     # "отсекаем" лишние части строки
outcome = ('{} ({})'.format(i1, i2))     # формируем итоговый вывод

answers['1'] = outcome     

display(answers['1'])

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
i = data[data.runtime == data.runtime.max()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['2'] = outcome     

display(answers['2'])

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
i = data[data.runtime == data.runtime.min()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['3'] = outcome     

display(answers['3'])

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [8]:
answers['4'] = round(data["runtime"].mean())

display(answers['4'])

110

# 5. Каково медианное значение длительности фильмов? 

In [9]:
answers['5'] = round(data["runtime"].median())

display(answers['5'])

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
# лучше код получения столбца profit вынести в Предобработку что в начале

i = data[data.profit == data.profit.max()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['6'] = outcome     

display(answers['6'])  

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [11]:
i = data[data.profit == data.profit.min()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['7'] = outcome     

display(answers['7'])  

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
answers['8'] = len(data[data.profit > 0])

display(answers['8'])  

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
i = data[data.revenue == data.query('release_year == 2008').revenue.max()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['9'] = outcome     

display(answers['9'])  

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [14]:
i = data[data.profit == data.query('2012<=release_year & release_year<=2014').profit.min()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['10'] = outcome     

display(answers['10'])

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [15]:
'''В нашем датафрейме один фильм может принадлежать к нескольким жанрам, 
перечисленным через разделитель '|', из-за чего мы не можем просто работать с колонкой 'genres'.'''

genres_list = data.genres.str.split('|').sum()  # заносим в общий список все жанры всех фильмов
genres_series = pd.Series(genres_list)          # создаем pd.Series из списка
genres_count = genres_series.value_counts()     # создаем упорядоченную по убыванию серию

answers['11'] = genres_count.index[0]           # первый (максималный) элемент будет ответом

display(answers['11'])


'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [16]:
'''Задача решается аналогично предыдущей, только для фильмов с прибылью ('profit') больше нуля'''

genres_list = data[data['profit'] > 0].genres.str.split('|').sum()
genres_series = pd.Series(genres_list) 
genres_count = genres_series.value_counts()

answers['12'] = genres_count.index[0] 

display(answers['12'])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

Вариант 1:

In [17]:
directors_list = data.director.str.split('|').sum()  # создаем список всех режиссеров

directors = []            # создаем список режиссеров и "очищаем" его от дубликатов
for i in directors_list:
    if i not in directors:
        directors.append(i)

director_revenue = {}     # создаем словарь куда будем вносить имя_режиссера:сумму_сборов
for i in directors:
    director_revenue[i] = data[data.director.str.contains(i)]['revenue'].sum()

    
# создаем pd.Series из словаря и выбираем элемент с максимальным значением
answers['13'] = pd.Series(director_revenue).idxmax() 

display(answers['13'])

'Peter Jackson'

Вариант 2:

In [18]:
# создаем список режиссеров с помощью .explode() и .tolist()
directors = data.director.str.split('|').explode().value_counts().index.tolist()
        
directors_revenue = Counter() # для подсчета суммы сборов применяем счетчик вместо словаря
for i in directors:
    directors_revenue[i] = data.loc[data['director'].str.contains(i)]['revenue'].sum()

answers['13'] = pd.Series(directors_revenue).idxmax() 

display(answers['13'])

'Peter Jackson'

Вариант 3:

In [19]:
directors = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

directors['director'] = directors.director.str.split('|') # преобразуем перечень режиссеров в список
directors=directors.explode('director')  # "разбиваем" список режиссеров для каждого фильма

# группируем режиссеров по сумме прибыли, упорядочив по убыванию
dirs_revenue = directors.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

answers['13'] = dirs_revenue.index[0] # первый (максималный) элемент будет ответом

display(answers['13'])

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [20]:
actions = data[data.genres.str.contains('Action')] # выбираем все экшены из датафрейма 

#создаем упорядоченную по убыванию серию с режиссерами
actions_directors = actions.director.str.split('|').explode().value_counts()

answers['14'] = actions_directors.index[0]        # первый (максималный) элемент будет ответом

display(answers['14'])

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

Вариант 1:

In [21]:
films_2012 = data[data['release_year'] == 2012].copy()  # выбираем фильмы 2012 года 

# создаем список с именами всех актеров из фильмов 2012 года
actors = films_2012.cast.str.split('|').explode().value_counts().index.tolist()

actors_revenue = Counter() # создаем счетчик в котором будем суммировать сборы для актеров
for i in actors:
    actors_revenue[i] = films_2012.loc[data['cast'].str.contains(i)]['revenue'].sum()

# создаем pd.Series из счетчика и выбираем 1й элемент (с максимальным значением)
answers['15'] = pd.Series(actors_revenue).idxmax() 

display(answers['15'])

'Chris Hemsworth'

Вариант 2:

In [22]:
films_2012 = data[data['release_year'] == 2012].copy()  # выбираем фильмы 2012 года 

films_2012['cast'] = films_2012.cast.str.split('|') # преобразуем перечень актеров в список
films_2012=films_2012.explode('cast')   # "разбиваем" список актеров для каждого фильма

# группируем актеров по сумме сборов, упорядочив по убыванию
actors_profit = films_2012.groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

answers['15'] = actors_profit.index[0] # первый (максималный) элемент будет ответом

display(answers['15'])

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?
#### Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [23]:
# создаем новый датафрейм с фильмами с бюджетом выше среднего
high_b_films = data[data['budget'] > data['budget'].mean()]  

#создаем упорядоченную по убыванию серию с актерами
high_b_actors = high_b_films.cast.str.split('|').explode().value_counts()

answers['16'] = high_b_actors.index[0]   # первый (максималный) элемент будет ответом

display(answers['16'])

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [24]:
# создаем новый датафрейм с фильмами с участием Николаса Кейджа
n_cage = data[data.cast.str.contains('Nicolas Cage')] 

genres_list = n_cage.genres.str.split('|').sum()  # заносим в общий список все жанры фильмов
genres_series = pd.Series(genres_list)            # создаем pd.Series из списка
genres_count = genres_series.value_counts()       # создаем упорядоченную по убыванию серию

answers['17'] = genres_count.index[0]             # первый (максималный) элемент будет ответом

display(answers['17'])

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [25]:
# создаем новый датафрейм с фильмами с от Paramount Pictures
paramount = data[data.production_companies.str.contains('Paramount Pictures')]

# выбираем фильм с наименьшей прибылью
i = paramount[paramount.profit == paramount.profit.min()]

title = str(i.original_title)
imdb = str(i.imdb_id)
i1 = title[title.find(' ') + 4 : title.find('\nName:')]
i2 = imdb[imdb.find(' ') + 4 : imdb.find('\nName:')]
outcome = ('{} ({})'.format(i1, i2))

answers['18'] = outcome     

display(answers['18'])  

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [26]:
# группируем года выпуска фильмов по сумме сборов, упорядочив по убыванию
years = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False) 

answers['19'] = years.index[0]    # первый (максималный) элемент будет ответом

display(answers['19'])  

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [27]:
# создаем новый датафрейм с фильмами с от Warner Bros
warner = data[data.production_companies.str.contains('Warner Bros')]

# группируем года выпуска фильмов по сумме прибыли, упорядочив по убыванию
years = warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

answers['20'] = years.index[0]    # первый (максималный) элемент будет ответом

display(answers['20'])  

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [28]:
# создаем список с месяцами релиза всех фильмов используя datetime.month
months = []
for i in range (len(data)):
    months.append(data['release_date'][i].month)

#создаем упорядоченную по убыванию серию с месяцами релиза
months_count = pd.Series(months).value_counts()

# первый (максималный) элемент будет ответом
answer = months_count.index[0]

# создаем список с "прогумерованными" названиями месяцев
month_names= pd.Series(["Январь", "Февраль", "Март", "Апрель", 
                        "Май", "Июнь", "Июль", "Август", 
                        "Сентябрь", "Октябрь", "Ноябрь", "Декабрь"],
                        index = range (1, 13))

# первый (максималный) элемент будет ответом
answers['21'] = month_names.loc[answer]    

display(answers['21'])

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [29]:
# для решения будем использовать серию months_count, созданную на предыдущем шаге

summer_films = 0
for i in range (6, 9):   # просуммируем кол-во фильмов вышедших в 6, 7 и 8 месяц
    summer_films += months_count.loc[i]
    
answers['22'] = summer_films

display(answers['22'])

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [30]:
# создаем запрос с перечислением зимних месяцев
q = 'release_date.dt.month == 1 | release_date.dt.month == 2 | release_date.dt.month == 12'

#создаем упорядоченную по убыванию серию с режиссерами
winter_directors = data.query(q).director.str.split('|').explode().value_counts()

answers['23'] = winter_directors.index[0]        # первый (максималный) элемент будет ответом

display(answers['23'])

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [31]:
names = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

# заменяем колонку с названиями фильмов на их длины
names['original_title'] = names.original_title.apply(lambda x: len(x))

# превращаем перечень студий в список
names['production_companies'] = names.production_companies.str.split('|')

# "разбиваем" список студий для каждого фильма
names = names.explode('production_companies')

# создаем упорядоченную по убыванию серию со средними длинами названий фильмов каждой студии
lengths = names.groupby(['production_companies'])['original_title'].mean()\
          .sort_values(ascending = False)

answers['24'] = lengths.index[0]  # первый (максималный) элемент будет ответом

display(answers['24'])

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [32]:
overviews = data.copy()    # создаем копию исходного датафрейма, с которой будем работать

# заменяем колонку с описаниями фильмов на кол-во слов в них
overviews['overview'] = overviews.overview.apply(lambda x: len(x.split()))

# превращаем перечень студий в список
overviews.production_companies = overviews.production_companies.str.split('|')

# "разбиваем" список студий для каждого фильма
overviews = overviews.explode('production_companies')

# создаем упорядоченную по убыванию серию со средними длинами описаний фильмов каждой студии
lengths = overviews.groupby(['production_companies'])['overview'].mean()\
          .sort_values(ascending = False)

answers['25'] = lengths.index[0]  # первый (максималный) элемент будет ответом

display(answers['25'])

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [33]:
# вычисляем 99% квантиль рейтинга фильмов
rate_99=data.quantile(0.99)['vote_average']

# делаем выборку фильмов с рейтингом выше 99% квантиля
best=data[data.vote_average > rate_99]

# в качестве ответа берем список названий фильмов из выборки
answers['26'] =  best.original_title.tolist()

display(answers['26'])

# В получившемся списке фильмов больше чем в вариантах ответа. Остается только
# выбрать вариант, все фильмы которого содержатся в нашем списке

['Inside Out',
 'Room',
 'Interstellar',
 'Guardians of the Galaxy',
 'The Imitation Game',
 'Gone Girl',
 'The Grand Budapest Hotel',
 'Inception',
 'The Dark Knight',
 'The Pianist',
 'The Lord of the Rings: The Return of the King',
 'The Wolf of Wall Street',
 '12 Years a Slave',
 'Memento']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [34]:
actors = data.copy()    # создаем копию исходного датафрейма, с которой будем работать
actors['cast'] = actors.cast.str.split('|')  # превращаем перечень актеров в список

actor_pairs = []        # создаем список куда будем заносить пары актеров

for i in range(len(actors)):
    actor_pairs.extend(combinations(actors.loc[i].cast, 2))
    
pairs_series = pd.Series(actor_pairs) 
pairs_count = pairs_series.value_counts()
pairs_df = pd.DataFrame(pairs_count, columns = ['counts'])
pairs_max = pairs_df[pairs_df.counts == pairs_df.counts.max()]    

# в качестве ответа берем список названий фильмов из выборки
answers['27'] =  pairs_max.index.tolist()

display(answers['27'])

# Мы получили 2 пары актеров, снимавшихся вместе больше других, остается выбрать подходящую

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

# Submission

In [35]:
# в конце можно посмотреть свои ответы к каждому вопросу
display(answers)

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out',
  'Room',
  'Interstellar',
  'Guardians of the Galaxy',
  'The Imitation Game',
  'Gone Girl',
  'The Grand Budapest Hotel',
  'Inception',
  'The Dark Knight',
  'The Pianist',
  'The Lord of the Rings: The Return of the King',
  'The Wolf of Wall Street',
  '12 Years a Slave',
  'Memento'],
 '27': [('Daniel Radcliffe', 'R

In [36]:
# и убедиться что ни чего не пропустил)
len(answers)

27